# In this code we compute the derivative of power spectrum wrt cosmo parameter and Mnu

cf article: for precision about numerical steps, fiducial values...
We compute a matrix of derivatives $\partial_\theta \mathbf{P}=(\frac{\partial P}{\partial \theta}(k_i,z_j))_{i,j}$

In the Fisher code, we will find the best couple $i,j$ for a given $k$ and $z$ (by dichotomy), and take the given coefficient of $\partial_\theta \mathbf{P}$

In [3]:
import numpy as np
import scipy as sp
#import pyccl as ccl
import math
import matplotlib.pyplot as plt
from scipy import integrate
from functools import partial
from scipy.integrate import quad, dblquad
import pyccl as ccl
import camb
from camb import model, initialpower

mnu_fid=0.06
coeff_nu=0.05

H0_fid=67.7
coeff_h0=0.01

ombh2_fid=0.02247
coeff_ombh=0.01

omch2_fid=0.11923
coeff_omch=0.01

tau_fid=0.0568
coeff_tau=0.5

ns_fid=0.9675
coeff_ns=0.01

As_fid=2.215e-9
coeff_As=0.01


npointp=1000
kminp=5e-4
kmaxp=0.5
hierarchy='normal'

pars = camb.CAMBparams()
pars.set_accuracy(AccuracyBoost=2)
pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
pars.InitPower.set_params(ns=ns_fid,As=As_fid)
zeffp=3
pars.set_matter_power(redshifts=[zeffp], kmax=10)
results= camb.get_results(pars)
k_fid,z_fid,p_fid=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp)




def power_camb(zeff):
    pars = camb.CAMBparams()
    pars.set_accuracy(AccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k_fid,z_fid,p_fid=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    return p_fid[0]



def DP_Dh0_5(zeff):
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid*(1+coeff_h0), ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,ph01=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid*(1+2*coeff_h0), ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,ph02=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid*(1-coeff_h0), ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,ph0m1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid*(1-2*coeff_h0), ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,ph0m2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    return [[(-ph02[0,i]+8*ph01[0,i]-8*ph0m1[0,i]+ph0m2[0,i])/(coeff_h0*H0_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    

def DP_Dombh_5(zeff):
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid*(1+coeff_ombh), omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid*(1+2*coeff_ombh), omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid*(1-coeff_ombh), omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid*(1-2*coeff_ombh), omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]

    return [[(-p2[0,i]+8*p1[0,i]-8*pm1[0,i]+pm2[0,i])/(coeff_ombh*ombh2_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    

def DP_Domch_5(zeff):
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid*(1+coeff_omch),mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid*(1+2*coeff_omch),mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid*(1-coeff_omch),mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid*(1-2*coeff_omch),mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    return [[(-p2[0,i]+8*p1[0,i]-8*pm1[0,i]+pm2[0,i])/(coeff_omch*omch2_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    

def DP_Dns_5(zeff):
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid*(1+coeff_ns),As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid*(1+2*coeff_ns),As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid*(1-coeff_ns),As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid*(1-2*coeff_ns),As=As_fid)
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    return [[(-p2[0,i]+8*p1[0,i]-8*pm1[0,i]+pm2[0,i])/(coeff_ns*ns_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    




def DP_DAs_5(zeff):
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid*(1+coeff_As))
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid*(1+2*coeff_As))
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,p2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid*(1-coeff_As))
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid*(1-2*coeff_As))
    pars.set_matter_power(redshifts=[zeff], kmax=10)
    results= camb.get_results(pars)
    k,z,pm2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    #plt.plot(k,[(-p2[0,i]+8*p1[0,i]-8*pm1[0,i]+pm2[0,i])/(coeff_As*As_fid*12) for i in range(len(k))])
    return [[(-p2[0,i]+8*p1[0,i]-8*pm1[0,i]+pm2[0,i])/(coeff_As*As_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    

def DP_Dnu_5(zeffp):
    #with no nu power spectrum
    
    hierarchy='degenerate'
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid*(1+coeff_nu),tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,pnu1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid*(1+2*coeff_nu),tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,pnu2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid*(1-1*coeff_nu),tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,pnum1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid*(1-2*coeff_nu),tau=tau_fid,neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,pnum2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    return [[(-pnu2[0,i]+8*pnu1[0,i]-8*pnum1[0,i]+pnum2[0,i])/(coeff_nu*mnu_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    

def DP_Dtau_5(zeffp):
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid*(1+coeff_tau),neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,p1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f1=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid*(1+2*coeff_tau),neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,p2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    f2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid*(1-coeff_tau),neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,pm1=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm1=results.get_fsigma8()[0]/results.get_sigma8()[0]

    pars = camb.CAMBparams()
    #pars.set_accuracy(AccuracyBoost=2)
    camb.model.TransferParams(high_precision=True,accurate_massive_neutrino=True)
    pars.set_accuracy(AccuracyBoost=2,lAccuracyBoost=2)
    pars.set_cosmology(H0=H0_fid, ombh2=ombh2_fid, omch2=omch2_fid,mnu=mnu_fid,tau=tau_fid*(1-2*coeff_tau),neutrino_hierarchy=hierarchy)
    pars.InitPower.set_params(ns=ns_fid,As=As_fid)
    pars.set_matter_power(redshifts=[zeffp], kmax=1)
    results= camb.get_results(pars)
    k,z,pm2=results.get_matter_power_spectrum(minkh=kminp,maxkh=kmaxp,npoints=npointp,var1=model.Transfer_nonu,var2=model.Transfer_nonu)
    fm2=results.get_fsigma8()[0]/results.get_sigma8()[0]
    
    return [[(-p2[0,i]+8*p1[0,i]-8*pm1[0,i]+pm2[0,i])/(coeff_tau*tau_fid*12) for i in range(len(k))],(-f2+8*f1-8*fm1+fm2)/(coeff_h0*H0_fid*12)]    


#%%
Zeff=[0.7,1.1,1.5,2,2.55,2.7,3.1,3.3,4.1]
A=[DP_Dh0_5(z) for z in Zeff]
dP_dh=[A[i][0] for i in range(len(Zeff))]
df_dh=[A[i][1] for i in range(len(Zeff))]
    
A=[DP_DAs_5(z) for z in Zeff]
dP_dAs=[A[i][0] for i in range(len(Zeff))]
df_dAs=[A[i][1] for i in range(len(Zeff))]


A=[DP_Dns_5(z) for z in Zeff]
dP_dns=[A[i][0] for i in range(len(Zeff))]
df_dns=[A[i][1] for i in range(len(Zeff))]

A=[DP_Dtau_5(z) for z in Zeff]
dP_dtau=[A[i][0] for i in range(len(Zeff))]
df_dtau=[A[i][1] for i in range(len(Zeff))]

A=[DP_Domch_5(z) for z in Zeff]
dP_domch=[A[i][0] for i in range(len(Zeff))]
df_domch=[A[i][1] for i in range(len(Zeff))]

A=[DP_Dombh_5(z) for z in Zeff]
dP_dombh=[A[i][0] for i in range(len(Zeff))]
df_dombh=[A[i][1] for i in range(len(Zeff))]

Pfid=[power_camb(z) for z in Zeff]

#%%
A=[DP_Dnu_5(z) for z in Zeff]
dP_dnu=[A[i][0] for i in range(len(Zeff))]
df_dnu=[A[i][1] for i in range(len(Zeff))]



Zeff.append(10) #usefull for later when checking the good z
print('end of derivative estimate')
#%%


end of derivative estimate


In [52]:
the_list=[dP_dnu,dP_dh,dP_dAs,dP_dns,dP_dtau,dP_domch,dP_dombh,]
string_list=['dP_dnu.txt','dP_dh.txt','dP_dAs.txt','dP_dns.txt','dP_dtau.txt','dP_domch.txt','dP_dombh.txt']
for i in range(len(the_list)):
    mat=np.matrix(the_list[i])
    with open(string_list[i],'wb') as f:
        for line in mat:
            np.savetxt(f, line, fmt='%12.6f')

In [38]:
the_list=[df_dnu,df_dh,df_dAs,df_dns,df_dtau,df_domch,df_dombh,]
string_list=['df_dnu.txt','df_dh.txt','df_dAs.txt','df_dns.txt','df_dtau.txt','df_domch.txt','df_dombh.txt']
for i in range(len(the_list)):
    mat=the_list[i]
    with open(string_list[i],'wb') as f:
        np.savetxt(f,mat, fmt='%12.6f')